<a href="https://colab.research.google.com/github/isaacmattern/wordle/blob/main/wordle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Wordle Solver**
### Python project by Isaac Mattern

[Wordle](https://www.powerlanguage.co.uk/wordle/) is a game where the objective is to correctly guess a 5-letter word in 6 tries or less. Each time a player submits a guess, the game highlights each letter in the word in

*   green, if the letter is in the word and in the correct location in the word
*   yellow, if the letter is in the word, but not in the position where the user guessed
*   gray, if the word does not contain the letter at all

This project is an attempt to use a list of the 12,972 5-letter words which are valid Wordle guesses and some Python magic to solve some Wordles.

I will be attempting to devise different algorithms to best solve Wordle.

In [66]:
# Control the amount of simulated games each algorithm will play
simulations = 100

##Getting Words
First thing's first, Wordle is all about 5-letter words. The Wordle site gives a list of all valid Wordle guesses (it's in the JavaScript for the site - the entire game is completely client-side!). We can just use those words to more accurately simulate Wordle instead of grabbing a list of English words which will contain many invalid guesses. 

In [59]:
# Grab the file we want from my github repo
!wget -q https://raw.githubusercontent.com/isaacmattern/wordle/main/valid-wordle-words.txt

In [88]:
words_file = open("valid-wordle-words.txt", "r")

all_words = []
for line in words_file:
  word = line.strip()
  if(len(word) == 5):
    all_words.append(word)

#### Answer List
The first 2315 words in our list are Wordle answers for the years 2021 to 2027, when the game will no longer have any answers (as of the time I'm writing this). The site gives the answers first, and then the rest of guesses, so we'll make a separate array called **all_answers** out of the first 2315 list elements.

In [89]:
all_answers = all_words[:2315]

## Defining Some Important Functions

Wordle will tell us certain information about our guess which we can use to eliminate many words from our list of possible words.

*   A **green** letter means we can eliminate any word which doesn't have the letter at the spot where we guessed it
*   A **yellow** letter means we can eliminate any words without that letter and any words with that letter in that exact position
*   A **gray** letter is a little more complicated. If the letter is gray, and that letter only occurred once in our guess, then we can eliminate any word which contains that letter. If our guess contained the letter multiple times, then we must see if any of the other occurrences were green or yellow. If all of them were gray, we can eliminate all words with that letter at all, but if one of them was green or yellow (not gray), then we can only eliminate words which have letters at that specific position. 

Let's define two functions which will be useful regardless of what algorithm we are using.


1.   An **update_possible** function will allow us to trim down our list of possible Wordle solutions.
2.   A **get_colors** function will allow us to simulate what Wordle's program does each time you submit a guess. Thus, this function will only be used when we're running simulations to test the efficiency of our word-guessing algorithms

In [91]:
def update_possible(guess, possible, colors):
  """
  Uses a list of colors (equal in length
  to the length of our words) to eliminate words
  which could not possibly be correct. 
  """
  for i in range(len(guess)):
    # GREEN
    if colors[i] == 0:
      # Eliminate all words which do not have a correct letter in a correct spot
      for word in possible[:]:
        if guess[i] != word[i]:
          possible.remove(word)
    # YELLOW      
    elif colors[i] == 1:
      # Eliminate all words which do not contain a correct letter and 
      # all words with that letter in that specific position
      for word in possible[:]:
        if guess[i] not in word or guess[i] == word[i]:
          possible.remove(word)
    # GRAY
    else:
      # It could be the case that the user guessed a word with a repeated
      # letter - if that's the case, we also need to check if that letter is
      # in the word at all.

      # First, let's just check if we have more than one index
      if guess.count(guess[i]) == 1:
        # It is the case that the letter is not in our answer
        for word in possible[:]:
          if guess[i] in word:
            possible.remove(word)
      else:
        # There is more than one occurence of the letter
        # Let's grab the indices
        indices = [j for j, x in enumerate(guess) if x == guess[i]]
        # Check if the letter is in the answer at all
        in_word = False
        for j in indices:
          if colors[j] != 2:
            in_word == True
        if in_word:
          # Simply remove word with the letter at that index
          for word in possible[:]:
            if guess[i] == word[i]:
              possible.remove(word)
        else:
          # Remove all words with the letter
          for word in possible[:]:
            if guess[i] in word:
              possible.remove(word)

  if guess in possible:
    possible.remove(guess)
  return possible

def get_colors(guess, answer) -> list:
  """
  Compares a guess to an answer and
  returns a list of numbers which signifies
  colors returned by a wordle guess.
  0 = Green
  1 = Yellow
  2 = Gray
  """
  colors = []
  for i in range(len(guess)):
    if guess[i] == answer[i]:
      colors.append(0)
    elif guess[i] in answer:
      colors.append(1)
    else:
      colors.append(2)

  return colors

## Approach 1: Random Guess

We will first randomly select a word using *random.choice*. A completely random guess probably isn't the greatest strategy, so we shouldn't expect an amazing result. 

After selecting a random 100 words from the list and running a simulation for each of them, it took, on average, 6.37 guesses, which is kind of garbage, since more than 6 guesses is considered a loss by Wordle. 

In [92]:
import random

def random_guess() -> int:
  # Set up answer, word list of possible answers, and generate our first guess
  answer = random.choice(all_answers)
  possible = all_words.copy()
  guess = random.choice(possible)
  num_guesses = 1

  # Randomly select a possible answer and use color information to eliminate
  # wrong solutions until we have found our word
  while guess != answer:
    print(f"Guess #{num_guesses}: {guess} (incorrect)")
    colors = get_colors(guess, answer)
    possible = update_possible(guess, possible, colors)
    guess = random.choice(possible)
    num_guesses = num_guesses + 1

  print(f"{num_guesses} guesses for the solution \"{guess}\"\n")
  # print(num_guesses)
  return num_guesses

In [ ]:
total_guesses = 0

for i in range(simulations):
  total_guesses = total_guesses + random_guess()

average = total_guesses / float (simulations)
print(f"Average # of guesses for random: {average}")

## Calculating Letter Distributions

The "best" Wordle guess is the one which could potentially cut down the amount of possible guesses the most. This is logically equivalent to the guess which is most likely to have the most amount of greens. Words with the letters which occur the most in the list are the best words to guess. We can recalculate these counts after each new guess with a **get_letter_occurrences** function, and use an **update_alphabet** function to update our alphabet (list of valid characters)

In [53]:
def get_letter_occurrences(alphabet, possible) -> dict:
  letters = {}

  for letter in alphabet:
    letters[letter] = 0

  for word in possible:
    for letter in list(set(word)):
      letters[letter] = letters[letter] + 1

  return letters

def update_alphabet(alphabet, guess, colors) -> list:
  new_alphabet = alphabet[:]
  for i in range(len(guess)):
    # Recall that a color of 2 means gray, which means this letter is
    # not part of the answer
    if colors[i] == 2:
      if guess[i] in new_alphabet:
        new_alphabet.remove(guess[i])
  return new_alphabet

## Approach 2: Guess Words With Most Common Letters

The function below, **get_common_letter_guess** uses our list of letters and their counts (I used a Python dict) to, letter by letter, assign a total "score" to each word in our remaining possible answers. 

Theoretically, the word with the max score should give us the most green squares, although there may be some flaw here: it might be the case that a letter is extremely common words, but uncommon in specific columns. We can come back to this with a third approach. 

In [8]:
def get_common_letter_guess(possible, letters) -> str:

  # Compute a score for each word
  best = {}
  for word in possible:
    best[word] = 0
    for letter in list(set(word)):
      best[word] = best[word] + letters[letter]

  # Guess the word with the best score
  return max(best, key=best.get)

In [65]:
def common_letters_guess() -> int:
  # Set up answer, word list of possible answers, and generate our first guess
  answer = random.choice(all_answers)
  possible = all_words.copy()
  alphabet = list("abcdefghijklmnopqrstuvwxyz")
  letters = get_letter_occurrences(alphabet, possible)
  guess = get_common_letter_guess(possible, letters)
  num_guesses = 1

  # Randomly select a possible answer and use color information to eliminate
  # wrong solutions until we have found our word
  while guess != answer:
    print(f"Guess #{num_guesses}: {guess} (incorrect)")
    colors = get_colors(guess, answer)
    possible = update_possible(guess, possible, colors)
    alphabet = update_alphabet(alphabet, guess, colors)
    letters = get_letter_occurrences(alphabet, possible)
    guess = get_common_letter_guess(possible, letters)
    num_guesses = num_guesses + 1

  print(f"{num_guesses} guesses for the solution \"{guess}\"\n")
  return num_guesses

In [ ]:
total_guesses = 0

for i in range(simulations):
  total_guesses = total_guesses + common_letters_guess()

average = total_guesses / float (simulations)
print(f"Average # of guesses for common letter guess: {average}")

## Analysis So Far:

Based on 100 simulations each,

Approach 1: Random Guessing takes an average of **5.04 guesses**

Approach 2: Guess words with most common letters takes an average of **4.11 guesses**

So, assigning a score to each word based off of how many words we can eliminate if the answer is completely wrong, and choosing the "best" word, we decrease our score, on average, by almost an entire guess. This is pretty impressive!